#**SQLite?**

[SQLite](https://fr.wikipedia.org/wiki/SQLite) est une bibliothèque écrite en langage C qui propose **un moteur de base de données relationnelle accessible par le langage SQL**.

Nous avons déjà détaillé dans le cours de ce thème le modèle relationnel.

Contrairement aux serveurs de bases de données traditionnels, comme MySQL (utilisé lors du TD SQL), sa particularité est de ne pas reproduire le schéma habituel client-serveur mais d'être directement intégrée aux programmes.

L'intégralité de la base de données (déclarations, tables, index et données) est stockée dans un fichier indépendant de la plateforme.

Dans ce TD, nous allons écrire des commandes en SQL avec SQLite et via Python.

## Créer une nouvelle Base de données

A la différence des tables qu'on utilise habituellement, la base de données n'est pas visible directement en ouvrant Excel ou un éditeur de texte via phpMyadmin.

Pour avoir une vue de ce que contient la base de données, il est nécessaire d'avoir un autre type de logiciel comme SQLLiteSpy.

In [19]:
import sqlite3
# Se connecter à une base de données SQL vide
# SQLite stocke la BDD dans un simple fichier
filepath = "./jeu.db"
open(filepath, 'w').close() #crée un fichier vide
conn = sqlite3.connect(filepath)


**La méthode [cursor](https://docs.python.org/3.5/library/sqlite3.html#sqlite3.Connection.cursor)** :

Il s'agit d'une sorte de tampon mémoire intermédiaire, destiné à mémoriser temporairement les données en cours de traitement, ainsi que les opérations que vous effectuez sur elles, avant leur transfert définitif dans la base de données.

Tant que la  méthode [commit](https://docs.python.org/3.5/library/sqlite3.html#sqlite3.Connection.commit) n'aura pas été appelée, aucun ordre ne sera appliqué à la base de données.

La connection à la base de données BDD est établie, créer une table auteur avec la requête SQL suivante:


In [20]:
# Déclaration du curseur c lié à la connection conn
c = conn.cursor()

# Créer une tables
c.execute('''CREATE TABLE IF NOT EXISTS stocks
             (id INTEGER PRIMARY KEY, date text, trans text, symbol text, qty real, price real)''')

# Insérer une ligne de données
c.execute("INSERT INTO stocks (date, trans, symbol, qty, price) VALUES ('2006-01-05','BUY','RHAT',100,35.14)")

# Sauvegarde des changements
conn.commit()


# Il est possible de cloturer la connection une fois les changements savegardés

#conn.close()


###**Afficher le contenue d'une table**###

Pour voir ce qu'il y a dans la table, on utilise un premier Select où on demande à voir toute la table

In [21]:

for row in c.execute('SELECT * FROM stocks '):
        print(row)

(1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


### **Insérer plusieurs données à la fois**

In [22]:
# Insérer plusieurs lignes
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks (date, trans, symbol, qty, price) VALUES (?,?,?,?,?)', purchases)
#t = ('IBM',)
#c.execute('SELECT * FROM stocks WHERE symbol=?', t)
c.execute('SELECT * FROM stocks')
print(c.fetchall()) # ou print(c.fetone()) pour afficher une ligne


[(1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14), (2, '2006-03-28', 'BUY', 'IBM', 1000.0, 45.0), (3, '2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0), (4, '2006-04-06', 'SELL', 'IBM', 500.0, 53.0)]


### Pandas
Nous pouvons utiliser la fonction read_sql_query (de la libraire Pandas) pour lire les résultats d’une requête SQL directement dans un DataFrame. Le code ci-dessous va exécuter la même requête que nous venons de faire, mais il retournera un DataFrame. Il a plusieurs avantages par rapport à la requête que nous avons faite ci-dessus:

Cela ne nous oblige pas à créer un objet Cursor ou à appeler fetchall à la fin.
Il lit automatiquement les noms des en-têtes de la table.
Il crée un DataFrame, afin que nous puissions rapidement explorer les données.

In [23]:
import pandas as pd

# méthode SQL Query
df1 = pd.read_sql_query('SELECT * FROM stocks', conn )
print("En utilisant la méthode read_sql_query \n", df1.head(), "\n")

#méthode DataFrame en utilisant la liste issue de .fetchall()
df2 = pd.DataFrame (df1, columns=[ 'date', 'trans', 'symbol', 'qty', 'price'])
print("En passant par une DataFrame \n", df2.head(),"\n")

# affichage de deux lignes sur la table
print(df2.head(2))

En utilisant la méthode read_sql_query 
    id        date trans symbol     qty  price
0   1  2006-01-05   BUY   RHAT   100.0  35.14
1   2  2006-03-28   BUY    IBM  1000.0  45.00
2   3  2006-04-05   BUY   MSFT  1000.0  72.00
3   4  2006-04-06  SELL    IBM   500.0  53.00 

En passant par une DataFrame 
          date trans symbol     qty  price
0  2006-01-05   BUY   RHAT   100.0  35.14
1  2006-03-28   BUY    IBM  1000.0  45.00
2  2006-04-05   BUY   MSFT  1000.0  72.00
3  2006-04-06  SELL    IBM   500.0  53.00 

         date trans symbol     qty  price
0  2006-01-05   BUY   RHAT   100.0  35.14
1  2006-03-28   BUY    IBM  1000.0  45.00


#**Activité 1**
En SQL, on utilise WHERE pour filtrer les tables selon certaines conditions.


1.   **Afficher** alors la liste des stocks ayant un "price" supérieur ou égale à 50.

In [24]:
# Saisir votre réponse ici
c.execute('SELECT * FROM stocks WHERE price>=50')
print(c.fetchall())

[(3, '2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0), (4, '2006-04-06', 'SELL', 'IBM', 500.0, 53.0)]


Avec Pandas, on peut utiliser plusieurs manières de faire :
    - avec un booléen
    - en utilisant la méthode 'query'

In [25]:
df2[df2['symbol'] == "IBM"]

,date,trans,symbol,qty,price
1,2006-03-28,BUY,IBM,1000.0,45.0
3,2006-04-06,SELL,IBM,500.0,53.0


In [26]:
df2.query('symbol == "IBM"')

,date,trans,symbol,qty,price
1,2006-03-28,BUY,IBM,1000.0,45.0
3,2006-04-06,SELL,IBM,500.0,53.0


Pour mettre plusieurs conditions, on utilise :
- & en Python, AND en SQL
- | en python, OR en SQL

2. **Afficher** alors en utilisant les deux méthodes les stocks ayant un prix supérieur à 50 et un symbol ='IBM'

In [27]:
# Saisir votre réponse ici
c.execute('SELECT * FROM stocks WHERE price>=50 AND symbol= "IBM"')
print(c.fetchall())

[(4, '2006-04-06', 'SELL', 'IBM', 500.0, 53.0)]


# **Activité 2**

En pandas, l'opération GROUP BY de SQL s'effectue avec une méthode similaire : [groupby](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.groupby.html)

Elle sert à regrouper des observations en groupes selon les modalités de certaines variables en appliquant une fonction d'aggrégation sur d'autres variables.

In [28]:
# Afficher alors avec une requête SQL
# chaque "symbol" associé à son occurance dans la table comme suit: [('IBM', 4), ('MSFT', 2), ('RHAT', 1)]

c.execute('SELECT symbol, id FROM stocks GROUP BY "symbol"')
print(c.fetchall())

[('IBM', 2), ('MSFT', 3), ('RHAT', 1)]


Attention, en pandas, la fonction [count](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.count.html?highlight=count#pandas.core.groupby.DataFrameGroupBy.count) ne fait pas la même chose qu'en SQL. [count](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.count.html?highlight=count#pandas.core.groupby.DataFrameGroupBy.count) s'applique à toutes les colonnes et compte toutes les observations non nulles.

In [29]:
df2.groupby('symbol').count()

,date,trans,qty,price
symbol,,,,
IBM,2,2,2,2
MSFT,1,1,1,1
RHAT,1,1,1,1


Pour réaliser la même chose qu'en SQL, il faut utiliser la méthode [size](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.core.groupby.DataFrameGroupBy.size.html?highlight=size#pandas.core.groupby.DataFrameGroupBy.size).

In [30]:
# Saisir par conséquent l'instruction avec size et exécuter...

c.execute('SELECT *, COUNT(*) AS size FROM stocks GROUP BY "symbol"')
print(c.fetchall())


[(2, '2006-03-28', 'BUY', 'IBM', 1000.0, 45.0, 2), (3, '2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0, 1), (1, '2006-01-05', 'BUY', 'RHAT', 100.0, 35.14, 1)]


In [31]:
# calculer maintenant le prix "price" moyen et le multiplier par 2

c.execute('SELECT price FROM stocks')
a = 0
valeurs = c.fetchall()
longueur = len(valeurs)
for i in range(len(valeurs)) :
    a = valeurs[i][0]+ a
a = (a/longueur)*2
print(a)

# Faites des recherches pour utiliser les méthodes apply et lambda

102.57


## Enregistrer une table SQL sous un autre format

On utilise le package csv, l'option 'w' pour 'write'.

On crée l'objet "writer", qui vient du package csv.

Cet objet a deux méthodes :
- writerow pour les noms de colonnes  : une liste
- writerows pour les lignes : un ensemble de liste


In [32]:
data = c.execute('SELECT * FROM stocks')

import csv

with open('./output.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow([]) #il faut saisir la liste des colonnes de la table
    writer.writerows(data)

On peut également passer par un DataFrame pandas et utiliser .to_csv().

A la fin de cette activité vous pouvez supprimer la table stocks et fermer la connection.

In [33]:
#Saisir votre réponse ici....
# proposer une solution avec .to_CSV()

df2.to_csv('data.csv')

# Supprimer la table et fermer la connection

conn.close()
import os
os.remove('data.csv')

# **Activité 3**



1.   Créer une base de données nommé VillesFrances avec SQLite
2.   Créer une connection et un curseur.

3.   Créer une table villes s'appuyant sur le modèle du fichier csv [data.gouv](https://www.data.gouv.fr/fr/datasets/r/dbe8a621-a9c4-4bc3-9cae-be1699c5ff25).
4.   Insérer au moins cinq lignes (villes de votre choix) de données dans la table et maximum 10 avec SQLite.
5.  Faites de même avec panda.

6.  Afficher avec des requêtes SQL les informations suivantes:

Faites les insertions de données nécessaires pour tester le fonctionnements de vos requêtes.

*   La liste des villes d'un même département avec leur longitude et lattitude.
*   Afficher la ville qui a une lattitude et une longitude maximalles


7. Créer une DataFrame avec Panda pour afficher les mêmes informations.


Commenter chaque solution.







In [34]:
# Saisir votre code ici

import sqlite3
open("./VillesFrances.db", "w").close()
conn = sqlite3.connect('VillesFrances.db')
c = conn.cursor()
c.execute("CREATE TABLE villes (code_commune_INSEE TEXT,nom_commune_postal TEXT,code_postal TEXT,libelle_acheminement TEXT,ligne_5 TEXT,latitude REAL,longitude REAL,code_commune TEXT,article TEXT,nom_commune TEXT,nom_commune_complet TEXT,code_departement TEXT,nom_departement TEXT,code_region TEXT,nom_region TEXT,PRIMARY KEY (code_commune_INSEE))")

#données des 5 premières lignes du fichier
ajouté = [
    ('1001', 'L ABERGEMENT CLEMENCIAT', '1400', 'L ABERGEMENT CLEMENCIAT', '', 46.1534255214, 4.92611354223, '1', "L'", 'Abergement-Clémenciat', "L'Abergement-Clémenciat", '1', 'Ain', '84', 'Auvergne-Rhône-Alpes'),
    ('1002', 'L ABERGEMENT DE VAREY', '1640', 'L ABERGEMENT DE VAREY', '', 46.0091878776, 5.42801696363, '2', "L'", 'Abergement-de-Varey', "L'Abergement-de-Varey", '1', 'Ain', '84', 'Auvergne-Rhône-Alpes'),
    ('1004', 'AMBERIEU EN BUGEY', '1500', 'AMBERIEU EN BUGEY', '', 45.9608475114, 5.3729257777, '4', '', 'Ambérieu-en-Bugey', 'Ambérieu-en-Bugey', '1', 'Ain', '84', 'Auvergne-Rhône-Alpes'),
    ('1005', 'AMBERIEUX EN DOMBES', '1330', 'AMBERIEUX EN DOMBES', '', 45.9961799872, 4.91227250796, '5', '', 'Ambérieux-en-Dombes', 'Ambérieux-en-Dombes', '1', 'Ain', '84', 'Auvergne-Rhône-Alpes'),
    ('1006', 'AMBLEON', '1300', 'AMBLEON', '', 45.7494989044, 5.59432017366, '6', '', 'Ambléon', 'Ambléon', '1', 'Ain', '84', 'Auvergne-Rhône-Alpes')
]

c.executemany('''INSERT INTO villes VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',ajouté)

#print ceux qui ont un code departemental de 1 
c.execute("SELECT nom_commune, latitude, longitude FROM villes WHERE code_departement = '1'")
print(c.fetchall())

#additionne latitude avec la longitude et on prend celui qui a la plus grosse somme
c.execute("SELECT nom_commune FROM villes WHERE latitude + longitude = (SELECT MAX(latitude + longitude) FROM villes)")
print(c.fetchall())

import pandas as pd

df = pd.read_sql_query("SELECT * FROM villes", conn)
df_dep_1 = df[df['code_departement'] == '1']
print(df_dep_1[['nom_commune', 'latitude', 'longitude']])

max_sum_row = df.iloc[(df['latitude'] + df['longitude']).idxmax()]
print(max_sum_row['nom_commune'])

conn.commit()
conn.close()

[('Abergement-Clémenciat', 46.1534255214, 4.92611354223), ('Abergement-de-Varey', 46.0091878776, 5.42801696363), ('Ambérieu-en-Bugey', 45.9608475114, 5.3729257777), ('Ambérieux-en-Dombes', 45.9961799872, 4.91227250796), ('Ambléon', 45.7494989044, 5.59432017366)]
[('Abergement-de-Varey',)]
             nom_commune   latitude  longitude
0  Abergement-Clémenciat  46.153426   4.926114
1    Abergement-de-Varey  46.009188   5.428017
2      Ambérieu-en-Bugey  45.960848   5.372926
3    Ambérieux-en-Dombes  45.996180   4.912273
4                Ambléon  45.749499   5.594320
Abergement-de-Varey
